![logo](https://github.com/donatellacea/DL_tutorials/blob/main/notebooks/figures/1128-191-max.png?raw=true)

# XAI in Deep Learning-Based Signal Analysis: Transformers

In this Notebook, we introduce the concept of transformers in machine learning, highlighting their significance in natural language processing.

---

## Getting Started

### Setup Colab environment

If you installed the packages and requirements on your own machine, you can skip this section and start from the import section.
Otherwise, you can follow and execute the tutorial on your browser. In order to start working on the notebook, click on the following button, this will open this page in the Colab environment and you will be able to execute the code on your own.

<a href="https://colab.research.google.com/github/HelmholtzAI-Consultants-Munich/Zero2Hero---Introduction-to-XAI/blob/Juelich-2023/data_and_models/Model-Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now that you opened the notebook in Colab, follow the next step:

1. Run this cell to connect your Google Drive to Colab and install packages
2. Allow this notebook to access your Google Drive files. Click on 'Yes', and select your account.
3. "Google Drive for desktop wants to access your Google Account". Click on 'Allow'.
   
At this point, a folder has been created in your Drive and you can navigate it through the lefthand panel in Colab, you might also have received an email that informs you about the access on your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
!git clone --branch Juelich-2023 https://github.com/HelmholtzAI-Consultants-Munich/XAI-Tutorials.git
%cd XAI-Tutorials/data_and_models

### Imports

In [1]:
import math
import torch
import torch.nn as nn

## 1. Encoder-Decoder Structure
 
Transformers are a type of neural network architecture that has become a cornerstone in the field of Natural Language Processing (NLP) and beyond. They were introduced in the 2017 paper "Attention is All You Need" by Vaswani et al. Transformers are a milestone because of the **Attention Mechanism**, especially self-attention. 

The Transformer architecture is divided into two main parts:

1. **Encoder:**
The encoder processes the input data (like a sentence in a translation task) and encodes it into a context-rich representation. It consists of a stack of layers, each containing a self-attention mechanism and a feed-forward neural network.

2. **Decoder:**
The decoder takes the encoded input and generates the output sequence (like a translated sentence). It also consists of a stack of layers, each with two attention mechanisms (one that attends to the output of the encoder and one that is a masked self-attention mechanism to prevent the decoder from seeing future tokens in the output sequence) and a feed-forward neural network.

<img src="..//docs/source/_figures/transformers.png" alt="transformers" width="800" height="500">

## 2. The Self-Attention Mechanism

Before digging into the encoder and decoder architecture, let's talk about THE most important concept of transformers: the self-attention mechanism.

The goal of self-attention is to generate a representation of each element in a sequence by considering the entire sequence. For example, in a sentence, the representation of a word is computed by attending to all words in the sentence, including the word itself. Here, the word "attending" refers to the process where the model determines how much focus or importance to assign to each element in the input sequence when computing a representation for a specific element. 

**Components**

The main components are the tree vectors **Queries** (Q), **Keys** (K), and **Values** (V), which are computed for each element in the input sequence. These are typically created by multiplying the input embeddings with three different weight matrices.

<img src="..//docs/source/_figures/qkv.png" alt="self-attention" width="700" height="700">

Q represents the current element in the sequence, used to compute attention scores against all keys. K Represents all elements in the sequence. The attention scores are calculated by the dot product of the query with each key, determining the level of attention each element receives. V is the actual representation of the sequence elements. The attention mechanism uses the computed scores to create a weighted sum of these values, forming the output for each element.

**Attention Scores**

The model calculates the attention score for each pair of elements in the sequence. This is typically done by taking the dot product of the query vector of one element with the key vector of another, which indicates how much focus to put on other parts of the input sequence when encoding a particular element.

<img src="..//docs/source/_figures/qkv1.png" alt="self-attention" width="700" height="650">

**Scaling and Normalization**

The dot product scores are scaled down (usually by the square root of the dimension of the key vectors), and a softmax function is applied to obtain the final attention weights. This normalization ensures that the weights across the sequence sum up to 1.

<img src="..//docs/source/_figures/qkv2.png" alt="self-attention" width="700" height="650">

**Weighted Sum**

The output for each element is then a weighted sum of the value vectors, where the weights are the attention scores. This results in a new representation for each element that incorporates information from the entire sequence.

<img src="..//docs/source/_figures/qkv3.png" alt="self-attention" width="700" height="700">

The overall mechanism can be summarized in the figure below.

<img src="..//docs/source/_figures/self-attention.png" alt="self-attention" width="700" height="800">

## 3. The Mechanism of Multi-Head Attention

In multi-head attention, the attention mechanism is run in parallel multiple times. Each parallel run is known as a "head."
Each head learns to pay attention to different parts of the input, allowing the model to capture various aspects of the information (like different types of syntactic or semantic relationships).

<img src="..//docs/source/_figures/multi-head-attention.png" alt="multi-head-attention" width="700" height="800">

The outputs of all attention heads are concatenated and then linearly transformed into the final output. This combination allows the model to pay attention to information from different representation subspaces at different positions.

<img src="..//docs/source/_figures/qkv4.png" alt="qkv4" width="700" height="600">

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

## 4. Input Processing

Before feeding the input into the transformer encoder, some steps are performed: 
- **Tokenization:** is a fundamental step in text processing and NLP. It involves splitting text into smaller units called tokens. Tokens are often words, but they can also be characters, subwords, or even sentences, depending on the level of tokenization. 
Example:
Text: "Natural Language Processing is fascinating."
Tokens: ["Natural", "Language", "Processing", "is", "fascinating"]

- **Input Embeddings:** The input sequence (e.g., a sentence) is converted into a sequence of vectors. This is done through embeddings which map words or tokens to high-dimensional vectors.

- **Positional Encodings:** Since the Transformer does not have recurrent or convolutional layers, it uses positional encodings to add information about the position of each token in the sequence. These positional encodings have the same dimension as the embeddings and are added to them.

In [3]:
# Define the positional encoding layer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]
    

## 5. The Encoder Part

The encoder in the Transformer architecture is designed to process and encode input sequences. It is a critical component for understanding and representing the input data in a form that the decoder can then use for tasks like translation or text generation. 
The encoder part is composed of:

- **Stack of Layers:** The encoder is composed of a stack of identical layers. The number of layers varies (e.g., the original Transformer model uses 6 layers), but each layer has the same structure.
- **Each Encoder Layer  ic composed of:**
  - A multi-head self-attention mechanism.
  - A position-wise fully connected feed-forward network: The position-wise feed-forward network applies two linear transformations with a ReLU activation in between each position individually. This component complements self-attention by processing each element (word or token) of the sequence independently, enriching the representation with individual element-level information.
  - Normalization and Residual Connections: After each sub-component (the self-attention and the feed-forward network), there is a process of normalization. Also, each sub-component has a residual connection around it. This means the output of each sub-component is added to its input, and then normalized. These features help in training deep networks.

The output of the final encoder layer is a sequence of vectors representing the input sequence. This output is then used as the input for the Transformer decoder.

In [4]:
# The position-wise fully connected feed-forward network
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

# Define the encoder layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

## 6. The Decoder Part

In the Transformer model, the decoder is structured similarly to the encoder but with key differences for output generation tasks like machine translation and text summarization:

- **Structure:** It consists of multiple layers, each with two types of multi-head attention mechanisms (self-attention and encoder-decoder attention) and a feed-forward neural network.

- **Input Processing:** The input to the decoder is the right-shifted target sequence, ensuring each position's prediction is based only on preceding elements.

- **Masked Self-Attention:** This prevents the decoder from accessing future positions in the sequence, which is crucial for maintaining sequential dependency.

- **Encoder-Decoder Attention:** Each layer in the decoder focuses on relevant parts of the encoder output, which is crucial for aligning the input and output sequences in tasks like translation.

- **Output Generation:** The decoder's top layer output goes through a linear layer and a softmax to generate word probabilities for the sequence.

- **Training Mechanism:** Utilizes "teacher forcing," where the actual output sequence is used as the next input, aiding in training efficiency but potentially causing exposure bias during inference.

In [5]:
# Define the decoder layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

## 7. Put It All Together

In [6]:
# Define the transformer model
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [8]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

In [9]:
transformer

Transformer(
  (encoder_embedding): Embedding(5000, 512)
  (decoder_embedding): Embedding(5000, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList(
    (0-5): 6 x DecoderLayer(
 

## References

[1] Vaswani A, Shazeer N, Parmar N, et al. Attention is All You Need[C]. Proceedings of the 31st International Conference on Neural Information Processing Systems. 2017: 6000–6010.

[2] http://jalammar.github.io/illustrated-transformer/